In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from torchvision.models import resnet50, ResNet50_Weights
import torch
from torchvision import datasets, transforms, utils
from torch import nn


In [2]:
from torchvision.models import resnet50, ResNet50_Weights
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim

# Load ResNet50 with random initialization
res1 = resnet50(weights=None)
res1.fc = nn.Linear(res1.fc.in_features, 10)  # Update output layer for 10 classes

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),               # Resize to 224x224
    transforms.Grayscale(3),                     # Convert 1 channel to 3 channels
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x + torch.rand(x.shape)/255),  # Dequantize pixel values
    transforms.Lambda(lambda x: (x - 0.5) * 2.0)  # Map from [0,1] -> [-1,1]
])

# Load dataset
batch_size = 32
dataloader_train = torch.utils.data.DataLoader(
    datasets.MNIST('./mnist_data', download=True, train=True, transform=transform),
    batch_size=batch_size,
    shuffle=True
)


# Select device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = res1.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)

#Training loop
num_epochs = 1
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for inputs, labels in dataloader_train:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the device
        optimizer.zero_grad()  # Clear gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader_train):.4f}')


Epoch [1/1], Loss: 0.1625


In [3]:
torch.save(model.state_dict(), "10epochresnet11222.pt")


In [4]:
model = resnet50(weights=None)
model.fc = nn.Linear(model.fc.in_features, 10)  # Update output layer for 10 classes

model.load_state_dict(torch.load( "10epochresnet1.pt"))
model = model.to(device)


/tmp/ipykernel_62780/934244244.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load( "10epochresnet1.pt"))


In [10]:
#from torcheval.metrics.functional import multiclass_f1_score
import matplotlib.pyplot as plt
transform = transforms.Compose([
    transforms.Resize((224, 224)),               # Resize to 224x224
    transforms.Grayscale(3),                     # Convert 1 channel to 3 channels
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x + torch.rand(x.shape)/255),  # Dequantize pixel values
    transforms.Lambda(lambda x: (x - 0.5) * 2.0)  # Map from [0,1] -> [-1,1]
])
batch_size =32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataloader_test = torch.utils.data.DataLoader(
    datasets.MNIST('./mnist_data', download=True, train=False, transform=transform),
    batch_size=batch_size,
    shuffle=True
)


acc_history =[]
def test(epoch):
    model.eval() # set model in inference mode (need this because of dropout)
    test_loss = 0
    correct = 0
    
    for data, target in dataloader_test:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(dataloader_test) # loss function already averages over batch size
    accuracy = 100. * correct / len(dataloader_test.dataset)
    acc_history.append(accuracy)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(dataloader_test.dataset),
        accuracy))
test(1)


Test set: Average loss: 0.0000, Accuracy: 9910/10000 (99%)



In [6]:
import numpy as np
dummy =torch.FloatTensor([np.zeros((3,224, 224))]).to(device)

/tmp/ipykernel_62780/2323534067.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  dummy =torch.FloatTensor([np.zeros((3,224, 224))]).to(device)


In [7]:

softmax = nn.Softmax(dim=1)
a = softmax(model(dummy))

In [8]:
from numpy import asarray
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import exp
 

 
# calculate the inception score for p(y|x)
def inceptionscore(p_yx, eps=1E-16):
	# calculate p(y)
	p_y = expand_dims(p_yx.mean(axis=0), 0)
	# kl divergence for each image
	kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
	# sum over classes
	sum_kl_d = kl_d.sum(axis=1)
	# average over images
	avg_kl_d = mean(sum_kl_d)
	# undo the logs
	is_score = exp(avg_kl_d)
	return is_score
 
# conditional probabilities for high quality images
p_yx = asarray([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
score = inceptionscore(p_yx)
print(score)

2.999999999999999


In [9]:
inceptionscore(a.cpu().detach().numpy())    

1.0